# Evaluation of Spider 2.0 Snowflake Queries
## For each query, expect this format:
1. The Question / Context for the Snowflake Query
2. The Gold "Answer" Query from Spider 2.0 ./evaluation_suite/gold/sql
3. Lifebloom's "Answer" Query from the same starter question.
4. Verdict and Notes
## Replication
From a Snowflake account, you can replicate these exact results by uploading this notebook into your Snowflake account - all of the data sources here are (currently) free and openly available via the Snowflake Marketplace from vendors like CYBERSYN, Yes Energy, Braze, and others.


## SF-001
### Question: Assuming today is April 1, 2024, I would like to know the daily snowfall amounts greater than 6 inches for each U.S. postal code during the week ending after the first two full weeks of the previous year. Show the postal code, date, and snowfall amount.

In [ ]:
WITH timestamps AS
(   
    SELECT
        DATE_TRUNC(year,DATEADD(year,-1,DATE '2024-08-29')) AS ref_timestamp,
        LAST_DAY(DATEADD(week,2 + CAST(WEEKISO(ref_timestamp) != 1 AS INTEGER),ref_timestamp),week) AS end_week,
        DATEADD(day, day_num - 7, end_week) AS date_valid_std
    FROM
    (   
        SELECT
            ROW_NUMBER() OVER (ORDER BY SEQ1()) AS day_num
        FROM
            TABLE(GENERATOR(rowcount => 7))
    ) 
)
SELECT
    country,
    postal_code,
    date_valid_std,
    tot_snowfall_in 
FROM 
    GLOBAL_WEATHER__CLIMATE_DATA_FOR_BI.standard_tile.history_day
NATURAL INNER JOIN
    timestamps
WHERE
    country='US' AND
    tot_snowfall_in > 6.0 
ORDER BY 
    postal_code,date_valid_std
;

In [ ]:
SELECT POSTAL_CODE, DATE_VALID_STD, TOT_SNOWFALL_IN
FROM GLOBAL_WEATHER__CLIMATE_DATA_FOR_BI.STANDARD_TILE.HISTORY_DAY
WHERE DATE_VALID_STD BETWEEN '2023-01-15' AND '2023-01-21'
  AND TOT_SNOWFALL_IN > 6
  AND COUNTRY = 'US'
ORDER BY DATE_VALID_STD, POSTAL_CODE;

## Verdict: Correct!

## SF-002
### As of December 31, 2022, which top 10 active large banks, each with assets over $10 billion, had the highest percentage of uninsured assets based on quarterly estimates?

In [ ]:
WITH big_banks AS (
    SELECT id_rssd
    FROM FINANCE__ECONOMICS.CYBERSYN.financial_institution_timeseries
    WHERE variable = 'ASSET'
      AND date = '2022-12-31'
      AND value > 1E10
)
SELECT name
FROM FINANCE__ECONOMICS.CYBERSYN.financial_institution_timeseries AS ts
INNER JOIN FINANCE__ECONOMICS.CYBERSYN.financial_institution_attributes AS att ON (ts.variable = att.variable)
INNER JOIN FINANCE__ECONOMICS.CYBERSYN.financial_institution_entities AS ent ON (ts.id_rssd = ent.id_rssd)
INNER JOIN big_banks ON (big_banks.id_rssd = ts.id_rssd)
WHERE ts.date = '2022-12-31'
  AND att.variable_name = '% Insured (Estimated)'
  AND att.frequency = 'Quarterly'
  AND ent.is_active = True
ORDER BY (1 - value) DESC
LIMIT 10;

In [ ]:
WITH ActiveLargeBanks AS ( 
    SELECT t.ID_RSSD, MAX(t.DATE) AS DATE, SUM(t.VALUE) AS Total_Assets 
    FROM FINANCE__ECONOMICS.CYBERSYN.FINANCIAL_INSTITUTION_TIMESERIES t 
    JOIN FINANCE__ECONOMICS.CYBERSYN.FINANCIAL_INSTITUTION_ENTITIES e 
    ON t.ID_RSSD = e.ID_RSSD 
    WHERE e.IS_ACTIVE = TRUE 
    AND t.DATE = '2022-12-31' 
    AND t.VARIABLE_NAME LIKE '%Assets%' 
    GROUP BY t.ID_RSSD 
    HAVING SUM(t.VALUE) > 10000000000 ), 
UninsuredAssets AS ( 
    SELECT u.ID_RSSD, u.DATE, SUM(u.VALUE) AS Uninsured_Assets 
    FROM FINANCE__ECONOMICS.CYBERSYN.FINANCIAL_INSTITUTION_TIMESERIES u 
    WHERE u.DATE = '2022-12-31' 
    AND u.VARIABLE_NAME = 'Deposits: Amount ($) Deposit Accounts of More Than $250,000' 
    AND u.ID_RSSD IN (
        SELECT ID_RSSD 
        FROM ActiveLargeBanks) 
        GROUP BY u.ID_RSSD, u.DATE ), 
RankedUninsuredPercent AS ( 
    SELECT a.ID_RSSD, a.Total_Assets, u.Uninsured_Assets, (u.Uninsured_Assets / a.Total_Assets) * 100 AS Uninsured_Percentage 
    FROM ActiveLargeBanks a 
    JOIN UninsuredAssets u 
    ON a.ID_RSSD = u.ID_RSSD ) 
SELECT 
    e.NAME, r.ID_RSSD, r.Total_Assets, r.Uninsured_Assets, 
    r.Uninsured_Percentage 
FROM RankedUninsuredPercent r 
JOIN FINANCE__ECONOMICS.CYBERSYN.FINANCIAL_INSTITUTION_ENTITIES e 
ON e.ID_RSSD = r.ID_RSSD 
ORDER BY r.Uninsured_Percentage DESC 
LIMIT 10;

## Verdict:  Incorrect.

## SF-003
### From 2015 to 2020, which zip code in Census Zip Code Tabulation Areas had the second-highest annual population growth rate, given a minimum estimate of 25,000 people over a 5-year period? Include the zip code, state abbreviation, and growth rate.

In [ ]:
WITH zip_stats AS (
    SELECT
        YEAR(ts.date) AS year,
        ts.geo_id AS zip,
        rship.related_geo_name AS state,
        ts.value AS population,
        LAG(ts.value, 1) OVER (PARTITION BY zip ORDER BY year ASC) AS prev_year_population,
        population / prev_year_population - 1 AS pct_growth,
        population - prev_year_population AS absolute_change
    FROM GLOBAL_GOVERNMENT.CYBERSYN.american_community_survey_timeseries AS ts
    JOIN GLOBAL_GOVERNMENT.CYBERSYN.american_community_survey_attributes AS att
        ON ts.variable = att.variable
    JOIN GLOBAL_GOVERNMENT.CYBERSYN.geography_index AS geo
        ON ts.geo_id = geo.geo_id
    JOIN GLOBAL_GOVERNMENT.CYBERSYN.geography_relationships AS rship
        ON ts.geo_id = rship.geo_id AND rship.related_level = 'State'
    WHERE
        att.series_type = 'Total Population'
        AND att.measurement_type = 'Estimate'
        AND att.measurement_period = '5YR'
        AND geo.level = 'CensusZipCodeTabulationArea'
        AND ts.value > 25000
)

SELECT
    year,
    zip,
    state,
    pct_growth AS growth_percentage
FROM (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY year ORDER BY pct_growth DESC NULLS LAST) AS annual_rank
    FROM zip_stats
    WHERE year BETWEEN 2015 AND 2020
)
WHERE annual_rank = 2
ORDER BY year;

In [ ]:
SELECT 'Further data sourcing is necessary for completion of user objectives.' AS RESOLUTION_NOTICE;

## Verdict: Incorrect!

## SF-006
### What is the change ratio in the number of active financial branch entities in each state from before March 1, 2020, to the end of 2021?

In [ ]:
WITH pre_covid AS (
    SELECT state_abbreviation,
           COUNT(*) AS pre_covid_count
    FROM FINANCE__ECONOMICS.CYBERSYN.financial_branch_entities
    WHERE start_date <= '2020-03-01'
      AND (end_date >= '2020-03-01' OR end_date IS NULL)
    GROUP BY state_abbreviation
)
SELECT cur.state_abbreviation,
       pre_covid_count,
       COUNT(*) AS current_count,
       (current_count / CAST(pre_covid_count AS FLOAT)) - 1 AS pct_change
FROM FINANCE__ECONOMICS.CYBERSYN.financial_branch_entities AS cur
INNER JOIN pre_covid ON (cur.state_abbreviation = pre_covid.state_abbreviation)
WHERE (end_date IS NULL OR end_date >= '2021-12-31')
GROUP BY cur.state_abbreviation, pre_covid_count
ORDER BY pct_change DESC;


In [ ]:
WITH Active_Branches_Pre_March_2020 AS (
    SELECT 
        STATE_ABBREVIATION, 
        COUNT(*) AS Active_Branches_Before_March_2020
    FROM 
        FINANCE__ECONOMICS.CYBERSYN.FINANCIAL_BRANCH_ENTITIES
    WHERE 
        IS_ACTIVE = TRUE
        AND START_DATE < '2020-03-01'
        AND (END_DATE IS NULL OR END_DATE >= '2020-03-01')
        AND STATE_ABBREVIATION IS NOT NULL
    GROUP BY 
        STATE_ABBREVIATION
),
Active_Branches_End_2021 AS (
    SELECT 
        STATE_ABBREVIATION, 
        COUNT(*) AS Active_Branches_End_Of_2021
    FROM 
        FINANCE__ECONOMICS.CYBERSYN.FINANCIAL_BRANCH_ENTITIES
    WHERE 
        IS_ACTIVE = TRUE
        AND START_DATE <= '2021-12-31'
        AND (END_DATE IS NULL OR END_DATE >= '2021-12-31')
        AND STATE_ABBREVIATION IS NOT NULL
    GROUP BY 
        STATE_ABBREVIATION
)
SELECT 
    COALESCE(p.STATE_ABBREVIATION, e.STATE_ABBREVIATION) AS STATE_ABBREVIATION,
    COALESCE(p.Active_Branches_Before_March_2020, 0) AS Active_Branches_Before_March_2020, 
    COALESCE(e.Active_Branches_End_Of_2021, 0) AS Active_Branches_End_Of_2021,
    CASE 
        WHEN COALESCE(p.Active_Branches_Before_March_2020, 0) = 0 THEN NULL
        ELSE ((COALESCE(e.Active_Branches_End_Of_2021, 0) - COALESCE(p.Active_Branches_Before_March_2020, 0)) / NULLIF(COALESCE(p.Active_Branches_Before_March_2020, 0), 0) * 100) 
    END AS Change_Ratio
FROM 
    Active_Branches_Pre_March_2020 p
FULL OUTER JOIN 
    Active_Branches_End_2021 e
ON 
    p.STATE_ABBREVIATION = e.STATE_ABBREVIATION
ORDER BY 
    STATE_ABBREVIATION;

## Verdict: Correct*
### We say correct, but it is unclear. The question clearly indicates that only active branches should be counted, but the "Gold standard" answer does not filter for active only, whereas the Lifebloom query does (among other differences)

## SF-008
### What is the percentage change in gross income inflow and purchase-only seasonally-adjusted home price index for the Phoenix-Mesa-Scottsdale, AZ Metro Area from the beginning to the end of 2023?

In [ ]:
WITH county_map AS (
    SELECT
        geo_id,
        geo_name,
        related_geo_id,
        related_geo_name
    FROM US_REAL_ESTATE.CYBERSYN.geography_relationships
    WHERE geo_name = 'Phoenix-Mesa-Scottsdale, AZ Metro Area'
    AND related_level = 'County'
), 
gross_income_data AS (
    SELECT
        geo_id,
        date,
        SUM(value) AS gross_income_inflow
    FROM US_REAL_ESTATE.CYBERSYN.irs_origin_destination_migration_timeseries AS ts
    JOIN county_map ON (county_map.related_geo_id = ts.to_geo_id)
    WHERE ts.variable_name = 'Adjusted Gross Income'
    GROUP BY geo_id, date
), 
home_price_data AS (
    SELECT LAST_DAY(date, 'year') AS end_date, AVG(value) AS home_price_index
    FROM US_REAL_ESTATE.CYBERSYN.fhfa_house_price_timeseries AS ts
    JOIN US_REAL_ESTATE.CYBERSYN.fhfa_house_price_attributes AS att
        ON (ts.variable = att.variable)
    WHERE geo_id IN (SELECT geo_id FROM county_map)
      AND att.index_type = 'purchase-only'
      AND att.seasonally_adjusted = TRUE
    GROUP BY end_date
),
combined_data AS (
    SELECT
        gid.date,
        gid.gross_income_inflow,
        hpi.home_price_index
    FROM gross_income_data AS gid
    JOIN home_price_data AS hpi ON (gid.date = hpi.end_date)
),
aggregated_data AS (
    SELECT
        MIN(date) AS first_year,
        MAX(date) AS last_year
    FROM combined_data
),
summary_data AS (
    SELECT
        first_year,
        last_year,
        first_income.gross_income_inflow AS first_year_income,
        last_income.gross_income_inflow AS last_year_income,
        first_index.home_price_index AS first_year_index,
        last_index.home_price_index AS last_year_index
    FROM aggregated_data AS ad
    JOIN combined_data AS first_income ON (first_income.date = ad.first_year)
    JOIN combined_data AS last_income ON (last_income.date = ad.last_year)
    JOIN combined_data AS first_index ON (first_index.date = ad.first_year)
    JOIN combined_data AS last_index ON (last_index.date = ad.last_year)
)
SELECT
    ((last_year_income - first_year_income) / first_year_income) * 100 AS income_growth_percent,
    ((last_year_index - first_year_index) / first_year_index) * 100 AS index_growth_percent
FROM summary_data;

In [ ]:
WITH hpi_data AS (
  SELECT DATE, VALUE
  FROM US_REAL_ESTATE.CYBERSYN.FHFA_HOUSE_PRICE_TIMESERIES
  WHERE GEO_ID = 'geoId/C38060' 
  AND VARIABLE_NAME = 'FHFA_HPI traditional purchase-only quarterly Seasonally Adjusted'
  AND DATE IN ('2023-03-31', '2023-12-31')
)
SELECT
  MAX(CASE WHEN DATE = '2023-12-31' THEN VALUE END) AS Ending_Value,
  MAX(CASE WHEN DATE = '2023-03-31' THEN VALUE END) AS Starting_Value,
  100 * (MAX(CASE WHEN DATE = '2023-12-31' THEN VALUE END) - MAX(CASE WHEN DATE = '2023-03-31' THEN VALUE END)) / MAX(CASE WHEN DATE = '2023-03-31' THEN VALUE END) AS HPI_Percentage_Change
FROM hpi_data;

## Verdict: Incorrect!

## SF-009
### A real estate company is looking for a comparison of the building types in Amsterdam and Rotterdam. They need to know the total surface area and the number of buildings for each type of building in both cities. Can you provide the building class and subclass, along with the total surface area and the number of buildings for both Amsterdam and Rotterdam?

In [ ]:
WITH BUILDING AS(
SELECT A.NAMES['local']::VARCHAR AS NAME
     ,B.Class
     ,B.subclass
     ,count(*) num_buidling
     ,SUM(B.surface_area_sq_m) sum_surface_area_sq_m
 FROM NETHERLANDS_OPEN_MAP_DATA.NETHERLANDS.V_ADMINISTRATIVE A
INNER
 JOIN(SELECT Class
            ,subclass
            ,surface_area_sq_m
            ,GEO_CORDINATES
        FROM NETHERLANDS_OPEN_MAP_DATA.NETHERLANDS.V_BUILDING
       WHERE SUBSTR(QUADKEY
             ,0
             ,8) IN ('12020210', '12020211')) B
   ON ST_COVERS(A.GEO_CORDINATES
            ,B.GEO_CORDINATES)
WHERE A.id IN('r324431@56'
            ,'r47811@69')
GROUP BY A.NAMES['local']
     ,B.Class
     ,B.subclass
ORDER BY B.class
     ,B.subclass)
SELECT CLASS,
SUBCLASS,
MAX(CASE WHEN NAME='Amsterdam' THEN sum_surface_area_sq_m END) AS SUM_SURFACE_AREA_SQ_M_AMSTERDAM,
MAX(CASE WHEN NAME='Rotterdam' THEN sum_surface_area_sq_m END) AS SUM_SURFACE_AREA_SQ_M_ROTTERDAM,
MAX(CASE WHEN NAME='Amsterdam' THEN num_buidling END) AS NUM_OF_BUILDING_AMSTERDAM,
MAX(CASE WHEN NAME='Rotterdam' THEN num_buidling END) AS NUM_OF_BUILDING_ROTTERDAM
FROM BUILDING
GROUP BY CLASS,SUBCLASS
ORDER BY CLASS,SUBCLASS;

In [ ]:
WITH AddressInspection AS (
  SELECT 
    id,
    class,
    subclass,
    surface_area_sq_m,
    f.value::string AS addr_string_key
  FROM 
    NETHERLANDS_OPEN_MAP_DATA.NETHERLANDS.V_BUILDING,
    LATERAL FLATTEN(input => PARSE_JSON(contains_addresses)) AS f
),
CityCategorization AS (
  SELECT 
    id,
    class,
    subclass,
    surface_area_sq_m,
    addr_string_key,
    CASE 
      WHEN addr_string_key LIKE '%"addr:city":"Amsterdam"%' THEN 'Amsterdam'
      WHEN addr_string_key LIKE '%"addr:city":"Rotterdam"%' THEN 'Rotterdam'
      ELSE NULL
    END AS city
  FROM 
    AddressInspection
)
SELECT 
  city,
  class,
  subclass,
  COUNT(id) AS building_count,
  SUM(surface_area_sq_m) AS total_surface_area
FROM 
  CityCategorization
WHERE 
  city IS NOT NULL
GROUP BY 
  city, class, subclass
ORDER BY 
  city, class, subclass;

## Verdict: Correct*
### We think this one is certainly debatable, but we need more details from the Spider 2.0 team to understand more about these specific QUADKEY/Administrative ID values were selected ('r324431@56','r47811@69').

## SF-010
### What are the cumulative percentages of mortgages near default in California for each recorded date in 2023, including those 90 to 180 days past due, in forbearance, or in the process of foreclosure, bankruptcy, or deed in lieu?

In [ ]:
SELECT
    ts.date,
    SUM(ts.value) AS pct_near_default
FROM
    US_REAL_ESTATE.CYBERSYN.fhfa_mortgage_performance_timeseries AS ts
    JOIN US_REAL_ESTATE.CYBERSYN.fhfa_mortgage_performance_attributes AS att ON (ts.variable = att.variable)
    JOIN US_REAL_ESTATE.CYBERSYN.geography_index AS geo ON (geo.geo_id = ts.geo_id)
WHERE
    att.variable_group IN (
        'Percent 90 to 180 Days Past Due Date',
        'Percent in Forbearance',
        'Percent in the Process of Foreclosure, Bankruptcy, or Deed in Lieu'
    )
    AND att.market = 'All Mortgages'
    AND geo.geo_name = 'California'
    AND YEAR(ts.date) = 2023
GROUP BY
    ts.date
ORDER BY
    ts.date;

In [ ]:
WITH filtered_data AS (
  SELECT 
    GEO_ID,
    VARIABLE_NAME,
    DATE,
    VALUE
  FROM 
    US_REAL_ESTATE.CYBERSYN.FHFA_MORTGAGE_PERFORMANCE_TIMESERIES
  WHERE 
    GEO_ID = 'geoId/06' AND
    DATE >= '2023-01-01' AND 
    DATE <= '2023-12-31' AND 
    (VARIABLE_NAME LIKE '%90 to 180 Days Past Due Date%' OR
     VARIABLE_NAME LIKE '%in Forbearance%' OR
     VARIABLE_NAME LIKE '%Foreclosure%')
)
SELECT 
  DATE, 
  SUM(VALUE) AS Cumulative_Percentage_Near_Default
FROM 
  filtered_data
GROUP BY 
  DATE
ORDER BY 
  DATE;

## Verdict: Incorrect!

## SF011
### Determine the population distribution within each block group relative to its census tract in New York State using 2021 ACS data. Include block group ID, census value, state county tract ID, total tract population, and the population ratio of each block group.

In [ ]:
 WITH TractPop AS (
    SELECT
        CG."BlockGroupID",
        FCV."CensusValue",
        CG."StateCountyTractID",
        CG."BlockGroupPolygon"
    FROM
        CENSUS_GALAXY__ZIP_CODE_TO_BLOCK_GROUP_SAMPLE.PUBLIC."Dim_CensusGeography" CG
    JOIN
        CENSUS_GALAXY__ZIP_CODE_TO_BLOCK_GROUP_SAMPLE.PUBLIC."Fact_CensusValues_ACS2021" FCV
        ON CG."BlockGroupID" = FCV."BlockGroupID"
    WHERE
        CG."StateAbbrev" = 'NY'
        AND FCV."MetricID" = 'B01003_001E'
),

TractGroup AS (
    SELECT
        CG."StateCountyTractID",
        SUM(FCV."CensusValue") AS "TotalTractPop"
    FROM
        CENSUS_GALAXY__ZIP_CODE_TO_BLOCK_GROUP_SAMPLE.PUBLIC."Dim_CensusGeography" CG
    JOIN
        CENSUS_GALAXY__ZIP_CODE_TO_BLOCK_GROUP_SAMPLE.PUBLIC."Fact_CensusValues_ACS2021" FCV
        ON CG."BlockGroupID" = FCV."BlockGroupID"
    WHERE
        CG."StateAbbrev" = 'NY'
        AND FCV."MetricID" = 'B01003_001E'
    GROUP BY
        CG."StateCountyTractID"
)

SELECT
    TP."BlockGroupID",
    TP."CensusValue",
    TP."StateCountyTractID",
    TG."TotalTractPop",
    CASE WHEN TG."TotalTractPop" <> 0 THEN TP."CensusValue" / TG."TotalTractPop" ELSE 0 END AS "BlockGroupRatio"
FROM
    TractPop TP
JOIN
    TractGroup TG
    ON TP."StateCountyTractID" = TG."StateCountyTractID";

In [ ]:
SELECT 'Process concluded: Objective cannot be fully achieved with current data, further data sourcing recommended.' AS Data_Exploration_Conclusion;

## Verdict: Incorrect!

## SF-012
### What were the total amounts of building and contents damage reported under the National Flood Insurance Program in the City of New York for each year from 2010 to 2019?

In [ ]:
SELECT 
    YEAR(claims.date_of_loss)               AS year_of_loss,
    claims.nfip_community_name,
    SUM(claims.building_damage_amount) AS total_building_damage_amount,
    SUM(claims.contents_damage_amount) AS total_contents_damage_amount
FROM WEATHER__ENVIRONMENT.CYBERSYN.fema_national_flood_insurance_program_claim_index claims
WHERE 
    claims.nfip_community_name = 'City Of New York' 
    AND year_of_loss >=2010 AND year_of_loss <=2019
GROUP BY year_of_loss, claims.nfip_community_name
ORDER BY year_of_loss, claims.nfip_community_name;

In [ ]:
SELECT YEAR(DATE_OF_LOSS) AS claim_year, 
       SUM(BUILDING_DAMAGE_AMOUNT) AS total_building_damage, 
       SUM(CONTENTS_DAMAGE_AMOUNT) AS total_contents_damage
FROM WEATHER__ENVIRONMENT.CYBERSYN.FEMA_NATIONAL_FLOOD_INSURANCE_PROGRAM_CLAIM_INDEX
WHERE NFIP_COMMUNITY_NAME = 'City Of New York'
AND DATE_OF_LOSS BETWEEN '2010-01-01' AND '2019-12-31'
GROUP BY claim_year
ORDER BY claim_year;

## Verdict: Correct!

## SF-013
### Determine the total length of roads for each class and subclass in Amsterdam and Rotterdam, based on specific QUADKEY segments '12020210' and '12020211'? Show the class, subclass, and total road lengths for both cities

In [ ]:
WITH ROAD AS(
SELECT A.NAMES['local']::VARCHAR AS NAME
     ,R.Class
     ,R.subclass
     ,SUM(R.length_m) AS SUM_OF_LENGTH
 FROM NETHERLANDS_OPEN_MAP_DATA.NETHERLANDS.V_ADMINISTRATIVE A
INNER
 JOIN(SELECT Class
            ,subclass
            ,length_m
            ,GEO_CORDINATES
        FROM NETHERLANDS_OPEN_MAP_DATA.NETHERLANDS.V_ROAD 
       WHERE SUBSTR(QUADKEY
             ,0
             ,8) IN ('12020210', '12020211')) R
   ON ST_COVERS(A.GEO_CORDINATES
            ,R.GEO_CORDINATES)
WHERE A.id IN('r324431@56'
            ,'r47811@69')
GROUP BY A.NAMES['local']
     ,R.Class
     ,R.subclass
ORDER BY R.class
     ,R.subclass)
SELECT CLASS,
SUBCLASS,
MAX(CASE WHEN NAME='Amsterdam' THEN SUM_OF_LENGTH END) AS AMSTERDAM,
MAX(CASE WHEN NAME='Rotterdam' THEN SUM_OF_LENGTH END) AS ROTTERDAM
FROM ROAD
GROUP BY CLASS,SUBCLASS
ORDER BY CLASS,SUBCLASS;

In [ ]:
SELECT
    V_ROAD.CLASS AS class,
    V_ROAD.SUBCLASS AS subclass,
    SUM(V_ROAD.LENGTH_M) AS total_length_meters
FROM
    NETHERLANDS_OPEN_MAP_DATA.NETHERLANDS.V_ROAD
WHERE
    LEFT(V_ROAD.QUADKEY, 8) IN ('12020210', '12020211')
GROUP BY
    V_ROAD.CLASS,
    V_ROAD.SUBCLASS
ORDER BY
    total_length_meters DESC;

## Verdict: Incorrect!

## SF-014
### What is the New York State ZIP code with the highest number of commuters traveling over one hour, according to 2021 ACS data? Include the total commuters, state benchmark for this duration, and state population.

In [ ]:
WITH Commuters AS (
    SELECT
        GE."ZipCode",
        SUM(CASE WHEN M."MetricID" = 'B08303_013E' THEN F."CensusValueByZip" ELSE 0 END +
            CASE WHEN M."MetricID" = 'B08303_012E' THEN F."CensusValueByZip" ELSE 0 END) AS "Num_Commuters_1Hr_Travel_Time"
    FROM
        CENSUS_GALAXY__AIML_MODEL_DATA_ENRICHMENT_SAMPLE.PUBLIC."LU_GeographyExpanded" GE
    JOIN
        CENSUS_GALAXY__AIML_MODEL_DATA_ENRICHMENT_SAMPLE.PUBLIC."Fact_CensusValues_ACS2021_ByZip" F
        ON GE."ZipCode" = F."ZipCode"
    JOIN
        CENSUS_GALAXY__AIML_MODEL_DATA_ENRICHMENT_SAMPLE.PUBLIC."Dim_CensusMetrics" M
        ON F."MetricID" = M."MetricID"
    WHERE
        GE."PreferredStateAbbrev" = 'NY'
        AND (M."MetricID" = 'B08303_013E' OR M."MetricID" = 'B08303_012E') -- Metric IDs for commuters with 1+ hour travel time
    GROUP BY
        GE."ZipCode"
),

StateBenchmark AS (
    SELECT
        SB."StateAbbrev",
        SUM(SB."StateBenchmarkValue") AS "StateBenchmark_Over1HrTravelTime",
        SB."TotalStatePopulation"
    FROM
        CENSUS_GALAXY__AIML_MODEL_DATA_ENRICHMENT_SAMPLE.PUBLIC."Fact_StateBenchmark_ACS2021" SB
    WHERE
        SB."MetricID" IN ('B08303_013E', 'B08303_012E')
        AND SB."StateAbbrev" = 'NY'
    GROUP BY
        SB."StateAbbrev", SB."TotalStatePopulation"
)

SELECT
    C."ZipCode",
    SUM(C."Num_Commuters_1Hr_Travel_Time") AS "Total_Commuters_1Hr_Travel_Time",
    SB."StateBenchmark_Over1HrTravelTime",
    SB."TotalStatePopulation",
FROM
    Commuters C
CROSS JOIN
    StateBenchmark SB
GROUP BY
    C."ZipCode", SB."StateBenchmark_Over1HrTravelTime", SB."TotalStatePopulation"
ORDER BY
    "Total_Commuters_1Hr_Travel_Time" DESC
LIMIT 1;




In [ ]:
WITH ValidCommute AS (
    SELECT 
        VARIABLE, VARIABLE_NAME, SERIES_ID, MEASURE
    FROM 
        GLOBAL_GOVERNMENT.CYBERSYN.AMERICAN_COMMUNITY_SURVEY_ATTRIBUTES
    WHERE 
        VARIABLE_NAME ILIKE '%60 to 89 minutes%' AND
        MEASUREMENT_PERIOD = '1YR'
)
SELECT DISTINCT
    VC.VARIABLE, VC.VARIABLE_NAME, VC.MEASURE
FROM
    ValidCommute VC
LIMIT 10;

## Verdict: Incorrect!

## SF-018
### Examine user engagement with push notifications within a specified one-hour window on June 1, 2023.

In [ ]:
WITH push_send AS (
    SELECT
        id,
        app_group_id,
        user_id,
        campaign_id,
        message_variation_id,
        platform,
        ad_tracking_enabled,
        TO_TIMESTAMP(TIME) AS "TIME",
        'Send' AS "EVENT_TYPE"
    FROM
        BRAZE_USER_EVENT_DEMO_DATASET.PUBLIC.USERS_MESSAGES_PUSHNOTIFICATION_SEND_VIEW
    WHERE
        TO_TIMESTAMP(TIME) BETWEEN '2023-06-01 08:00:00' AND '2023-06-01 09:00:00'
),
push_bounce AS (
    SELECT
        id,
        app_group_id,
        user_id,
        campaign_id,
        message_variation_id,
        platform,
        ad_tracking_enabled,
        TO_TIMESTAMP(TIME) AS "TIME",
        'Bounce' AS "EVENT_TYPE"
    FROM
        BRAZE_USER_EVENT_DEMO_DATASET.PUBLIC.USERS_MESSAGES_PUSHNOTIFICATION_BOUNCE_VIEW
    WHERE
        TO_TIMESTAMP(TIME) BETWEEN '2023-06-01 08:00:00' AND '2023-06-01 09:00:00'
),
push_open AS (
    SELECT
        id,
        app_group_id,
        user_id,
        campaign_id,
        message_variation_id,
        platform,
        ad_tracking_enabled,
        TO_TIMESTAMP(TIME) AS "TIME",
        'Open' AS "EVENT_TYPE",
        carrier,
        browser,
        device_model
    FROM
        BRAZE_USER_EVENT_DEMO_DATASET.PUBLIC.USERS_MESSAGES_PUSHNOTIFICATION_OPEN_VIEW
    WHERE
        TO_TIMESTAMP(TIME) BETWEEN '2023-06-01 08:00:00' AND '2023-06-01 09:00:00'
),
push_open_influence AS (
    SELECT
        id,
        app_group_id,
        user_id,
        campaign_id,
        message_variation_id,
        platform,
        TO_TIMESTAMP(TIME) AS "TIME",
        'Influenced Open' AS "EVENT_TYPE",
        carrier,
        browser,
        device_model
    FROM
        BRAZE_USER_EVENT_DEMO_DATASET.PUBLIC.USERS_MESSAGES_PUSHNOTIFICATION_INFLUENCEDOPEN_VIEW
    WHERE
        TO_TIMESTAMP(TIME) BETWEEN '2023-06-01 08:00:00' AND '2023-06-01 09:00:00'
)
SELECT
    ps.app_group_id,
    ps.campaign_id,
    ps.user_id,
    ps.time,
    po.time push_open_time,
    ps.message_variation_id,
    ps.platform,
    ps.ad_tracking_enabled,
    po.carrier,
    po.browser,
    po.device_model,
    COUNT(
        DISTINCT ps.id
    ) push_notification_sends,
    COUNT(
        DISTINCT ps.user_id
    ) unique_push_notification_sends,
    COUNT(
        DISTINCT pb.id
    ) push_notification_bounced,
    COUNT(
        DISTINCT pb.user_id
    ) unique_push_notification_bounced,
    COUNT(
        DISTINCT po.id
    ) push_notification_open,
    COUNT(
        DISTINCT po.user_id
    ) unique_push_notification_opened,
    COUNT(
        DISTINCT poi.id
    ) push_notification_influenced_open,
    COUNT(
        DISTINCT poi.user_id
    ) unique_push_notification_influenced_open
FROM
    push_send ps
    LEFT JOIN push_bounce pb
    ON ps.message_variation_id = pb.message_variation_id
    AND ps.user_id = pb.user_id
    AND ps.app_group_id = pb.app_group_id
    LEFT JOIN push_open po
    ON ps.message_variation_id = po.message_variation_id
    AND ps.user_id = po.user_id
    AND ps.app_group_id = po.app_group_id
    LEFT JOIN push_open_influence poi
    ON ps.message_variation_id = poi.message_variation_id
    AND ps.user_id = poi.user_id
    AND ps.app_group_id = poi.app_group_id
GROUP BY
    1,2,3,4,5,6,7,8,9,10,11;


In [ ]:
WITH send_events AS (SELECT USER_ID, APP_GROUP_ID, CAMPAIGN_ID, MESSAGE_VARIATION_ID, PLATFORM, AD_TRACKING_ENABLED, COUNT(*) AS push_notification_sends FROM BRAZE_USER_EVENT_DEMO_DATASET.PUBLIC.USERS_MESSAGES_PUSHNOTIFICATION_SEND_VIEW WHERE TO_TIMESTAMP(TIME) BETWEEN '2023-06-01 08:00:00' AND '2023-06-01 09:00:00' GROUP BY USER_ID, APP_GROUP_ID, CAMPAIGN_ID, MESSAGE_VARIATION_ID, PLATFORM, AD_TRACKING_ENABLED), bounce_events AS (SELECT USER_ID, APP_GROUP_ID, CAMPAIGN_ID, MESSAGE_VARIATION_ID, PLATFORM, AD_TRACKING_ENABLED, COUNT(*) AS push_notification_bounced FROM BRAZE_USER_EVENT_DEMO_DATASET.PUBLIC.USERS_MESSAGES_PUSHNOTIFICATION_BOUNCE_VIEW WHERE TO_TIMESTAMP(TIME) BETWEEN '2023-06-01 08:00:00' AND '2023-06-01 09:00:00' GROUP BY USER_ID, APP_GROUP_ID, CAMPAIGN_ID, MESSAGE_VARIATION_ID, PLATFORM, AD_TRACKING_ENABLED), open_events AS (SELECT USER_ID, APP_GROUP_ID, CAMPAIGN_ID, MESSAGE_VARIATION_ID, PLATFORM, AD_TRACKING_ENABLED, CARRIER, BROWSER, DEVICE_MODEL, COUNT(*) AS push_notification_open FROM BRAZE_USER_EVENT_DEMO_DATASET.PUBLIC.USERS_MESSAGES_PUSHNOTIFICATION_OPEN_VIEW WHERE TO_TIMESTAMP(TIME) BETWEEN '2023-06-01 08:00:00' AND '2023-06-01 09:00:00' GROUP BY USER_ID, APP_GROUP_ID, CAMPAIGN_ID, MESSAGE_VARIATION_ID, PLATFORM, AD_TRACKING_ENABLED, CARRIER, BROWSER, DEVICE_MODEL), influenced_open_events AS (SELECT USER_ID, APP_GROUP_ID, CAMPAIGN_ID, MESSAGE_VARIATION_ID, PLATFORM, CARRIER, BROWSER, DEVICE_MODEL, COUNT(*) AS push_notification_influenced_open FROM BRAZE_USER_EVENT_DEMO_DATASET.PUBLIC.USERS_MESSAGES_PUSHNOTIFICATION_INFLUENCEDOPEN_VIEW WHERE TO_TIMESTAMP(TIME) BETWEEN '2023-06-01 08:00:00' AND '2023-06-01 09:00:00' GROUP BY USER_ID, APP_GROUP_ID, CAMPAIGN_ID, MESSAGE_VARIATION_ID, PLATFORM, CARRIER, BROWSER, DEVICE_MODEL ) SELECT COALESCE(s.USER_ID, b.USER_ID, o.USER_ID, ip.USER_ID) AS USER_ID, COALESCE(s.APP_GROUP_ID, b.APP_GROUP_ID, o.APP_GROUP_ID, ip.APP_GROUP_ID) AS APP_GROUP_ID, COALESCE(s.CAMPAIGN_ID, b.CAMPAIGN_ID, o.CAMPAIGN_ID, ip.CAMPAIGN_ID) AS CAMPAIGN_ID, COALESCE(s.MESSAGE_VARIATION_ID, b.MESSAGE_VARIATION_ID, o.MESSAGE_VARIATION_ID, ip.MESSAGE_VARIATION_ID) AS MESSAGE_VARIATION_ID, COALESCE(s.PLATFORM, b.PLATFORM, o.PLATFORM, ip.PLATFORM) AS PLATFORM, s.AD_TRACKING_ENABLED, o.CARRIER, o.BROWSER, o.DEVICE_MODEL, SUM(s.push_notification_sends) AS push_notification_sends, SUM(b.push_notification_bounced) AS push_notification_bounced, SUM(o.push_notification_open) AS push_notification_open, SUM(ip.push_notification_influenced_open) AS push_notification_influenced_open FROM send_events s FULL JOIN bounce_events b ON s.USER_ID = b.USER_ID AND s.CAMPAIGN_ID = b.CAMPAIGN_ID AND s.APP_GROUP_ID = b.APP_GROUP_ID AND s.PLATFORM = b.PLATFORM FULL JOIN open_events o ON b.USER_ID = o.USER_ID AND b.CAMPAIGN_ID = o.CAMPAIGN_ID AND b.APP_GROUP_ID = o.APP_GROUP_ID AND b.PLATFORM = o.PLATFORM FULL JOIN influenced_open_events ip ON o.USER_ID = ip.USER_ID AND o.CAMPAIGN_ID = ip.CAMPAIGN_ID AND o.APP_GROUP_ID = ip.APP_GROUP_ID AND o.PLATFORM = ip.PLATFORM AND o.CARRIER = ip.CARRIER AND o.BROWSER = ip.BROWSER AND o.DEVICE_MODEL = ip.DEVICE_MODEL GROUP BY COALESCE(s.USER_ID, b.USER_ID, o.USER_ID, ip.USER_ID), COALESCE(s.APP_GROUP_ID, b.APP_GROUP_ID, o.APP_GROUP_ID, ip.APP_GROUP_ID), COALESCE(s.CAMPAIGN_ID, b.CAMPAIGN_ID, o.CAMPAIGN_ID, ip.CAMPAIGN_ID), COALESCE(s.MESSAGE_VARIATION_ID, b.MESSAGE_VARIATION_ID, o.MESSAGE_VARIATION_ID, ip.MESSAGE_VARIATION_ID), COALESCE(s.PLATFORM, b.PLATFORM, o.PLATFORM, ip.PLATFORM), s.AD_TRACKING_ENABLED, o.CARRIER, o.BROWSER, o.DEVICE_MODEL LIMIT 10;

## Verdict: Incorrect

## SF-029
### Generate a daily detailed report for each product, showing sales units, revenue, average selling price (ASP), glance views, conversion rate, shipped units, shipped revenue, average net profit margin (PPM), and inventory details, starting 30 days prior to February 6, 2022, and including that date, for products under the 'Manufacturing' distributor view?

In [ ]:
select date
, asin
, max(sa.product_title)
, sum(sa.ordered_units) as ordered_units, sum(sa.ordered_revenue) as ordered_revenue
, sa.ordered_revenue/NULLIF(sa.ordered_units,0) as ASP
, sum(tr.glance_views) as glance_views
, CAST(sa.ordered_units as decimal(5,2))/NULLIF(tr.glance_views,0) as conversion_rate
, sum(sa.shipped_units) as shipped_units
, sum(sa.shipped_revenue) as shipped_revenue
, avg(net_ppm) as net_ppm
, avg(procurable_product_oos) as procurable_product_oos
, sum(sellable_on_hand_units) as sellable_on_hand_units
, sum(sellable_on_hand_inventory) as sellable_on_hand_value
, sum(net_received) as net_received_value
, sum(net_received_units) as net_received_units
, sum(open_purchase_order_quantity) as open_purchase_order_quantity
, sum(unfilled_customer_ordered_units) as unfilled_customer_ordered_units
, avg(vendor_confirmation_rate) as vendor_confirmation_rate
, avg(receive_fill_rate) as receive_fill_rate
, avg(sell_through_rate) as sell_through_rate
, avg(overall_vendor_lead_time_days) as vendor_lead_time_days

from AMAZON_VENDOR_ANALYTICS__SAMPLE_DATASET.PUBLIC.retail_analytics_sales sa
join AMAZON_VENDOR_ANALYTICS__SAMPLE_DATASET.PUBLIC.retail_analytics_traffic tr
using (date, asin, program, period, distributor_view)
join AMAZON_VENDOR_ANALYTICS__SAMPLE_DATASET.PUBLIC.retail_analytics_inventory inv
using (date, asin, program, period, distributor_view)
join AMAZON_VENDOR_ANALYTICS__SAMPLE_DATASET.PUBLIC.retail_analytics_net_ppm ppm
using (date, asin, program, period, distributor_view)

where date >= to_date('2022-02-06','YYYY-MM-DD') -30
and distributor_view = 'Manufacturing'
and period = 'DAILY' 

group by 1, 2, 6, 8
order by 1 desc


In [ ]:
WITH Traffic_CTE AS ( 
    SELECT DATE, ASIN, GLANCE_VIEWS 
    FROM AMAZON_VENDOR_ANALYTICS__SAMPLE_DATASET.PUBLIC.RETAIL_ANALYTICS_TRAFFIC 
    WHERE DISTRIBUTOR_VIEW = 'Manufacturing' 
    AND DATE BETWEEN '2022-01-07' AND '2022-02-06' ), 
Sales_CTE AS ( 
    SELECT DATE, ASIN, ORDERED_UNITS AS SALES_UNITS, SHIPPED_UNITS, SHIPPED_REVENUE 
    FROM AMAZON_VENDOR_ANALYTICS__SAMPLE_DATASET.PUBLIC.RETAIL_ANALYTICS_SALES 
    WHERE DISTRIBUTOR_VIEW = 'Manufacturing' 
    AND DATE BETWEEN '2022-01-07' AND '2022-02-06' ), 
Inventory_CTE AS ( 
    SELECT DATE, ASIN, SELLABLE_ON_HAND_INVENTORY 
    FROM AMAZON_VENDOR_ANALYTICS__SAMPLE_DATASET.PUBLIC.RETAIL_ANALYTICS_INVENTORY 
    WHERE DATE BETWEEN '2022-01-07' AND '2022-02-06' ), 
NetPPM_CTE AS ( 
    SELECT DATE, ASIN, NET_PPM 
    FROM AMAZON_VENDOR_ANALYTICS__SAMPLE_DATASET.PUBLIC.RETAIL_ANALYTICS_NET_PPM 
    WHERE DATE BETWEEN '2022-01-07' AND '2022-02-06' ) 
SELECT 
    t.DATE, t.ASIN, t.GLANCE_VIEWS, 
    IFNULL(s.SALES_UNITS, 0) AS SALES_UNITS, 
    IFNULL(s.SHIPPED_UNITS, 0) AS SHIPPED_UNITS, 
    IFNULL(s.SHIPPED_REVENUE, 0) AS SHIPPED_REVENUE, 
    ROUND(IFNULL(s.SALES_UNITS, 0) / NULLIF(t.GLANCE_VIEWS, 0), 4) AS CONVERSION_RATE, 
    CASE 
        WHEN s.SHIPPED_UNITS > 0 
        THEN ROUND(IFNULL(s.SHIPPED_REVENUE, 0) / s.SHIPPED_UNITS, 2) 
        ELSE NULL 
    END AS ASP, 
    i.SELLABLE_ON_HAND_INVENTORY, 
    n.NET_PPM 
FROM Traffic_CTE t 
LEFT JOIN Sales_CTE s ON t.ASIN = s.ASIN AND t.DATE = s.DATE 
LEFT JOIN Inventory_CTE i ON t.ASIN = i.ASIN AND t.DATE = i.DATE 
LEFT JOIN NetPPM_CTE n ON t.ASIN = n.ASIN AND t.DATE = n.DATE 
ORDER BY t.DATE, t.ASIN;

## Verdict: Incorrect*
### We were left a bit confused by this one, but alas our answer is certainly missing something and is not filtering on time the same way as the gold answer.

## SF-035
### How many unique users started sessions each day within each app group between June 1, 2023, and June 7, 2023? Also show the app group ID and the start day of the session.

In [ ]:
WITH session_joins AS (
    SELECT
        ss.app_group_id,
        ss.time AS ss_time,
        ss.user_id
    FROM
        BRAZE_USER_EVENT_DEMO_DATASET.PUBLIC.users_behaviors_app_sessionstart_view ss
        JOIN BRAZE_USER_EVENT_DEMO_DATASET.PUBLIC.users_behaviors_app_firstsession_view fs
        ON ss.user_id = fs.user_id
        AND ss.app_group_id = fs.app_group_id
    WHERE
        ss.time >= DATEDIFF(s, '1970-01-01', '2023-06-01') AND
        ss.time < DATEDIFF(s, '1970-01-01', '2023-06-08')
)
SELECT
    app_group_id,
    DATE_TRUNC('day', DATEADD(s, ss_time, '1970-01-01')) AS session_start_day,
    COUNT(
        DISTINCT user_id
    ) AS count_distinct_users
FROM
    session_joins
GROUP BY
    1, 2
ORDER BY
    session_start_day;

In [ ]:
SELECT 
  APP_GROUP_ID, 
  DATE_TRUNC('day', TO_TIMESTAMP(TIME)) AS session_date, 
  COUNT(DISTINCT USER_ID) AS unique_users
FROM 
  BRAZE_USER_EVENT_DEMO_DATASET.PUBLIC.USERS_BEHAVIORS_APP_SESSIONSTART_VIEW
WHERE 
  DATE_TRUNC('day', TO_TIMESTAMP(TIME)) BETWEEN '2023-06-01' AND '2023-06-07'
GROUP BY 
  APP_GROUP_ID, 
  session_date
ORDER BY 
  session_date, 
  APP_GROUP_ID;

## Verdict: Correct!

## SF-037
### For each Home Depot store identified by its POI ID, what is the distance in miles to the nearest Lowe's Home Improvement store?

In [ ]:
WITH joined_data AS (
    SELECT poi.poi_id, poi.poi_name, addr.longitude, addr.latitude,
           addr.number, addr.street_directional_prefix, addr.street,
           addr.street_type, addr.street_directional_suffix,
           addr.unit, addr.city, addr.state, addr.zip
    FROM US_REAL_ESTATE.CYBERSYN.point_of_interest_index AS poi
    JOIN US_REAL_ESTATE.CYBERSYN.point_of_interest_addresses_relationships AS map
        ON (poi.poi_id = map.poi_id)
    JOIN US_REAL_ESTATE.CYBERSYN.us_addresses AS addr
        ON (map.address_id = addr.address_id)
)
SELECT home_depot.poi_id,
       ST_DISTANCE(
           ST_MAKEPOINT(home_depot.longitude, home_depot.latitude),
           ST_MAKEPOINT(lowes.longitude, lowes.latitude)
       ) / 1609 AS distance_miles
FROM joined_data AS home_depot
JOIN joined_data AS lowes
WHERE home_depot.poi_name = 'The Home Depot'
  AND lowes.poi_name = 'Lowe''s Home Improvement'
QUALIFY ROW_NUMBER() OVER (PARTITION BY home_depot.poi_id ORDER BY distance_miles NULLS LAST) = 1;

In [ ]:
WITH home_depot AS (
    SELECT poi.POI_ID as HD_POI_ID, poi.POI_NAME as HD_POI_NAME, LATITUDE, LONGITUDE
    FROM US_ADDRESSES__POI.CYBERSYN.POINT_OF_INTEREST_INDEX poi
    JOIN US_ADDRESSES__POI.CYBERSYN.POINT_OF_INTEREST_ADDRESSES_RELATIONSHIPS rel ON poi.POI_ID = rel.POI_ID
    JOIN US_ADDRESSES__POI.CYBERSYN.US_ADDRESSES usa ON rel.ADDRESS_ID = usa.ADDRESS_ID
    WHERE POI_NAME = 'The Home Depot'
),
lowes AS (
    SELECT poi.POI_ID as LW_POI_ID, poi.POI_NAME as LW_POI_NAME, LATITUDE, LONGITUDE
    FROM US_ADDRESSES__POI.CYBERSYN.POINT_OF_INTEREST_INDEX poi
    JOIN US_ADDRESSES__POI.CYBERSYN.POINT_OF_INTEREST_ADDRESSES_RELATIONSHIPS rel ON poi.POI_ID = rel.POI_ID
    JOIN US_ADDRESSES__POI.CYBERSYN.US_ADDRESSES usa ON rel.ADDRESS_ID = usa.ADDRESS_ID
    WHERE POI_NAME = 'Lowe''s Home Improvement'
)
SELECT 
    hd.HD_POI_ID as Home_Depot_POI_ID,
    hd.HD_POI_NAME as Home_Depot_Name,
    lw.LW_POI_ID as Lowe_POI_ID,
    lw.LW_POI_NAME as Lowe_Name,
    3959 * acos(
        cos(radians(hd.LATITUDE)) 
        * cos(radians(lw.LATITUDE)) 
        * cos(radians(lw.LONGITUDE) - radians(hd.LONGITUDE)) 
        + sin(radians(hd.LATITUDE)) 
        * sin(radians(lw.LATITUDE))
    ) AS distance_miles
FROM home_depot hd
CROSS JOIN lowes lw
QUALIFY ROW_NUMBER() OVER (PARTITION BY hd.HD_POI_ID ORDER BY distance_miles ASC) = 1
ORDER BY hd.HD_POI_ID, distance_miles ASC;

## Verdict: Correct!

## SF-040
### Find the top 10 northernmost addresses in Florida's largest zip code area. What are their address numbers, street names, and types?

In [ ]:
WITH zip_areas AS (
    SELECT
        geo.geo_id,
        geo.geo_name AS zip,
        states.related_geo_name AS state,
        countries.related_geo_name AS country,
        ST_AREA(TRY_TO_GEOGRAPHY(value)) AS area
    FROM US_ADDRESSES__POI.CYBERSYN.geography_index AS geo
    JOIN US_ADDRESSES__POI.CYBERSYN.geography_relationships AS states
        ON (geo.geo_id = states.geo_id AND states.related_level = 'State')
    JOIN US_ADDRESSES__POI.CYBERSYN.geography_relationships AS countries
        ON (geo.geo_id = countries.geo_id AND countries.related_level = 'Country')
    JOIN US_ADDRESSES__POI.CYBERSYN.geography_characteristics AS chars
        ON (geo.geo_id = chars.geo_id AND chars.relationship_type = 'coordinates_geojson')
    WHERE geo.level = 'CensusZipCodeTabulationArea'
),

zip_area_ranks AS (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY country, state ORDER BY area DESC, geo_id) AS zip_area_rank
    FROM zip_areas
)

SELECT addr.number, addr.street, addr.street_type
FROM US_ADDRESSES__POI.CYBERSYN.us_addresses AS addr
JOIN zip_area_ranks AS areas
    ON (addr.id_zip = areas.geo_id)
WHERE addr.state = 'FL' AND areas.country = 'United States' AND areas.zip_area_rank = 1
ORDER BY LATITUDE DESC
LIMIT 10;

In [ ]:
SELECT "LATITUDE", "NUMBER", "STREET", COALESCE("STREET_TYPE", 'N/A') AS "STREET_TYPE"
FROM US_REAL_ESTATE.CYBERSYN.US_ADDRESSES
WHERE "ZIP" = '33852'
ORDER BY "LATITUDE" DESC
LIMIT 10;

## Verdict: Incorrect*
### We think that this is one that definitely comes down to phrasing. In our testing, models see this question as asking what is the highest population zip code - despite the question referencing "area."

## SF-041
### Generate a detailed analysis report for ERCOT on October 1, 2022, that integrates pricing, load forecasts, real-time load, and renewable generation data to provide insights into daily market dynamics and operational efficiency?

In [ ]:
with datetimes_ as ( --Grabbing standardized datetime data to which all data will be joined.
        select *
        from YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.iso_market_times_sample
        where iso = 'ERCOT'
            and datetime = '2022-10-01' -- Setting date specifically for October 1st, 2022.
),
    prices_ as (
        select *
        from YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.dart_prices_sample
        where iso = 'E' --ISO code (E = ERCOT).
            and objectid = 10000697078 --Price node unique numeric id.
),
    load_forc_ as ( --Load forecast data. To find datatypes, use the v_data_catalog_basic view to get the datatypeid and storage table in which the data is located.
        select *
        from YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.ts_forecast_sample
        where datatypeid = 19060
            and objectid = 10000712973
),
    load_ as ( --Real time load data.
        select *
        from YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.dart_loads_sample
        where objectid = 10000712973
),
    wind_gen_forc_ as ( --Wind generation forecast data.
        select *
        from YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.ts_forecast_sample
        where datatypeid = 9285
           and objectid = 10000712973
),
    wind_gen_ as ( --Real time wind data.
        select trunc(dateadd('second', 3599, datetime), 'hour') datetime --Converts the hourly rollup of 5 min data to hour ending (the 5 min are already interval ending themselves; just needs to be done for Snowflake).
              ,avg(value) wind_gen --Raw real time wind generation data is reported at the 5 min granularity.
        from YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.ts_gen_sample
        where datatypeid = 16
            and objectid = 10000712973
        group by all
),
    solar_gen_forc_ as ( --Solar generation forecast data.
        select *
        from YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.ts_forecast_sample
        where datatypeid = 662
            and objectid = 10000712973
),
    solar_gen_ as ( --Real time solar generation data.
        select trunc(dateadd('second', 3599, datetime), 'hour') datetime --Converts the hourly rollup of 5 min data to hour ending (the 5 min are already interval ending themselves; just needs to be done for Snowflake).
              ,avg(value) solar_gen --Raw real time solar generation data is reported at the 5 min granularity.
        from YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.ts_gen_sample
        where datatypeid = 650
            and objectid = 10000712973
        group by all
),
    combined_wide_data_ as ( --Combining the datatypes from above and grabbing only a subset of the columns. You can grab / "preserve" more columns such as the datatypeids from above by updating the code.
        select mt.iso --Datetime attributes.
              ,mt.datetime
              ,mt.timezone              
              ,mt.datetime_utc
              ,mt.onpeak
              ,mt.offpeak
              ,mt.wepeak
              ,mt.wdpeak
              ,mt.marketday
              ,ob1.objectname price_node_name --Object for the price data.
              ,pr.objectid price_node_id
              ,pr.dalmp
              ,pr.rtlmp
              ,ob2.objectname load_zone_name --Object for the ISO-wide load and generation data. You can also associate this object explicitly with each of the subsequent datatypes by updating the code.
              ,lf.objectid load_zone_id
              ,lf.value load_forecast
              ,lf.publishdate load_forecast_publish_date            
              ,ld.rtload
              ,wf.value wind_gen_forecast
              ,wf.publishdate wind_gen_forecast_publish_date
              ,wg.wind_gen
              ,sf.value solar_gen_forecast
              ,sf.publishdate solar_gen_forecast_publish_date
              ,sg.solar_gen
              ,load_forecast - wind_gen_forecast - solar_gen_forecast net_load_forecast --Easily calculate additional columns / features such as net load (load minus renewable generation).
              ,ld.rtload - wg.wind_gen - sg.solar_gen net_load_real_time --Second calculated feature for real time net load.
        from datetimes_ mt
        inner join prices_ pr
            on (mt.datetime = pr.datetime)
        inner join load_forc_ lf
            on (mt.datetime = lf.datetime)
        inner join load_ ld
            on (mt.datetime = ld.datetime)    
        inner join wind_gen_forc_ wf
            on (mt.datetime = wf.datetime)
        inner join wind_gen_ wg
            on (mt.datetime = wg.datetime)    
        inner join solar_gen_forc_ sf
            on (mt.datetime = sf.datetime)
        inner join solar_gen_ sg
            on (mt.datetime = sg.datetime)
        inner join YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.ds_object_list_sample ob1
            on (pr.objectid = ob1.objectid)
        inner join YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.ds_object_list_sample ob2
            on (lf.objectid = ob2.objectid)             
)
select *
from combined_wide_data_
order by datetime asc;


In [ ]:
WITH Aggregated_Renewable_Gen AS (
  SELECT 
    DATE_TRUNC('hour', DATETIME) AS RENEWABLE_HOUR,
    AVG(VALUE) AS AVG_RENEWABLE_GEN
  FROM 
    YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.TS_GEN_SAMPLE
  WHERE 
    DATE(DATETIME) = '2022-10-01'
  GROUP BY 
    DATE_TRUNC('hour', DATETIME)
),
Price_Load_Aggregated AS (
  SELECT 
    DATE_TRUNC('hour', PRICE_DATETIME) AS HOUR_TIMESTAMP,
    AVG(PRICE_LMP) AS AVG_PRICE_LMP,
    AVG(RT_LOAD) AS AVG_RT_LOAD
  FROM 
    (SELECT 
        rp.DATETIME AS PRICE_DATETIME,
        rp.LMP AS PRICE_LMP,
        rl.LOAD AS RT_LOAD
     FROM 
        YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.RT_PRICES_RECENT_SAMPLE rp
     JOIN 
        YES_ENERGY__SAMPLE_DATA.YES_ENERGY_SAMPLE.RT_LOADS_SAMPLE rl
     ON 
        rp.DATETIME = rl.DATETIME
     WHERE 
        DATE(rp.DATETIME) = '2022-10-01') pl
  GROUP BY DATE_TRUNC('hour', PRICE_DATETIME)
)
SELECT pa.HOUR_TIMESTAMP, pa.AVG_PRICE_LMP, pa.AVG_RT_LOAD, ar.AVG_RENEWABLE_GEN
FROM Price_Load_Aggregated pa
LEFT JOIN Aggregated_Renewable_Gen ar
ON pa.HOUR_TIMESTAMP = ar.RENEWABLE_HOUR
ORDER BY pa.HOUR_TIMESTAMP;

## Verdict: Correct*
### We could go either way on this, but given the vague wording of the question and the lack of additional materials provided - we would actually prefer the second one to be returned to our users.

## SF-044
### What is the percentage change in post-market close prices for the Magnificent 7 tech companies from January 1 to June 30, 2024?

In [ ]:
WITH ytd_performance AS (
  SELECT
    ticker,
    MIN(date) OVER (PARTITION BY ticker) AS start_of_year_date,
    FIRST_VALUE(value) OVER (PARTITION BY ticker ORDER BY date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS start_of_year_price,
    MAX(date) OVER (PARTITION BY ticker) AS latest_date,
    LAST_VALUE(value) OVER (PARTITION BY ticker ORDER BY date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS latest_price
  FROM FINANCE__ECONOMICS.CYBERSYN.stock_price_timeseries
  WHERE
    ticker IN ('AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA')
    AND date BETWEEN DATE '2024-01-01' AND DATE '2024-06-30'  -- Adjusted to cover only from the start of 2024 to the end of June 2024
    AND variable_name = 'Post-Market Close'
)
SELECT
  ticker,
  (latest_price - start_of_year_price) / start_of_year_price * 100 AS percentage_change_ytd
FROM
  ytd_performance
GROUP BY
  ticker, start_of_year_date, start_of_year_price, latest_date, latest_price
ORDER BY percentage_change_ytd DESC;

In [ ]:
-- Step 4: Calculate Percentage Change in Post-Market Close Prices
WITH price_changes AS (
    SELECT 
        TICKER, 
        MAX(CASE WHEN DATE = '2024-01-02' THEN VALUE END) AS START_PRICE,
        MAX(CASE WHEN DATE = '2024-06-28' THEN VALUE END) AS END_PRICE
    FROM (
        SELECT TICKER, DATE, VALUE
        FROM FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES
        WHERE VARIABLE_NAME = 'Post-Market Close'
        AND DATE IN ('2024-01-02', '2024-06-28')
        AND TICKER IN ('AAPL', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NVDA', 'TSLA')
    )
    GROUP BY TICKER
)
SELECT 
    TICKER,
    START_PRICE, 
    END_PRICE,
    ROUND(((END_PRICE - START_PRICE) / START_PRICE) * 100, 2) AS PERCENTAGE_CHANGE
FROM price_changes
ORDER BY TICKER;

## Verdict: Correct!

# Final Tally: 8 / 18
# Accuracy: 44.44%
An important caveat of course, is that the existing evaluation suite isn't possible to replicate since the Spider 2.0 team are not finished - and of course they are taking commentary on how to improve the questions/answers. That, combined with this purely using the Snowflake segment with N=18 examples doesn't fully scale - but we wanted to showcase what the Lifebloom model could do on the most difficult dataset out there!
